<h1><span style="color:gray">ipyrad-analysis toolkit:</span> tetrad</h1>

The `tetrad` tool is a framework for inferring a species tree topology using quartet-joining on large unlinked SNP data sets. It is particularly optimized for RAD-seq type datasets that are likely to involve a lot of missing data. 

### Load libraries

In [38]:
# conda install ipyrad -c conda-forge -c bioconda
# conda install tetrad -c conda-forge

In [2]:
import ipyrad.analysis as ipa
import toytree
import ipcoal

### Simulate a random tree with 20 tips and crown age of 10M generations

In [15]:
tree = toytree.rtree.bdtree(ntips=20, seed=555)
tree = tree.mod.node_scale_root_height(10e6) 
tree.draw(scalebar=True);

<svg class="toyplot-canvas-Canvas" height="360.0px" id="ta278a41b6e5d46d88a8df63527b63a4d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 360.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 r12 r13 r14 r15 r16 r17 r18 r19 0 5000000 10000000

### Simulate SNPs with missing data and write to database (.seqs.hdf5)

In [16]:
# init simulator with one diploid sample from each tip
model = ipcoal.Model(tree, Ne=1e6, nsamples=2, recomb=0)

# simulate sequence data on 10K loci
model.sim_loci(10000, 50)

# add missing data (50%)
model.apply_missing_mask(0.5)

# write results to database file
model.write_snps_to_hdf5(name="test-tet-miss50", outdir='/tmp', diploid=True)

wrote 259208 SNPs to /tmp/test-tet-miss50.snps.hdf5


### Infer tetrad tree

In [17]:
SNPS = "/tmp/test-tet-miss50.snps.hdf5"

In [19]:
tet = ipa.tetrad(
    data=SNPS,
    name="test-tet-miss50",
    workdir="/tmp",
    nboots=10, 
    nquartets=1e6,
)

loading snps array [20 taxa x 259208 snps]
max unlinked SNPs per quartet [nloci]: 10000
quartet sampler [full]: 4845 / 4845


In [20]:
tet.run(auto=True, force=True)

Parallel connection | latituba: 8 cores
initializing quartet sets database
[####################] 100% 0:00:14 | full tree * | mean SNPs/qrt: 3167 
[####################] 100% 0:00:12 | boot rep. 1 | mean SNPs/qrt: 3163 
[####################] 100% 0:00:12 | boot rep. 2 | mean SNPs/qrt: 3161 
[####################] 100% 0:00:12 | boot rep. 3 | mean SNPs/qrt: 3170 
[####################] 100% 0:00:12 | boot rep. 4 | mean SNPs/qrt: 3144 
[####################] 100% 0:00:11 | boot rep. 5 | mean SNPs/qrt: 3172 
[####################] 100% 0:00:11 | boot rep. 6 | mean SNPs/qrt: 3191 
[####################] 100% 0:00:11 | boot rep. 7 | mean SNPs/qrt: 3144 
[####################] 100% 0:00:11 | boot rep. 8 | mean SNPs/qrt: 3153 
[####################] 100% 0:00:11 | boot rep. 9 | mean SNPs/qrt: 3180 
[####################] 100% 0:00:11 | boot rep. 10 | mean SNPs/qrt: 3128 


### Draw the inferred tetrad tree

In [30]:
tre = toytree.tree(tet.trees.cons)
rtre = tre.root(["r19", "r18", "r17"])
rtre.draw(ts='d', use_edge_lengths=False, node_labels="support");

<svg class="toyplot-canvas-Canvas" height="360.0px" id="t2117e0dac77847e6b57f17bdfd096e54" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 360.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 100 100 70 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 r3 r2 r0 r1 r4 r5 r7 r6 r8 r9 r11 r10 r12 r13 r15 r14 r16 r18 r17 r19

### Does this tree match the *true* species tree?

In [37]:
rfdist = rtre.treenode.robinson_foulds(tree.treenode)[0]
rfdist == 0

True